In [1]:
#https://github.com/vwxyzjn/cleanrl/blob/master/cleanrl/c51_atari_visual.py
import numpy as np
from collections import deque
import gym
from gym import spaces
import cv2
import tqdm
cv2.ocl.setUseOpenCL(False)

In [2]:
class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        """Sample initial states by taking random number of no-ops on reset.
        No-op is assumed to be action 0.
        """
        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(1, self.noop_max + 1) #pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

In [3]:
class FireResetEnv(gym.Wrapper):
    def __init__(self, env):
        """Take action on reset for environments that are fixed until firing."""
        gym.Wrapper.__init__(self, env)
        assert env.unwrapped.get_action_meanings()[1] == 'FIRE'
        assert len(env.unwrapped.get_action_meanings()) >= 3

    def reset(self, **kwargs):
        self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(1)
        if done:
            self.env.reset(**kwargs)
        obs, _, done, _ = self.env.step(2)
        if done:
            self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)

In [4]:
class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        """Make end-of-life == end-of-episode, but only reset on true game over.
        Done by DeepMind for the DQN and co. since it helps value estimation.
        """
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done  = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condition for a few frames
            # so it's important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        """Reset only when lives are exhausted.
        This way all states are still reachable even though lives are episodic,
        and the learner need not know about any of this behind-the-scenes.
        """
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs

In [5]:
class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        """Return only every `skip`-th frame"""
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros((2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip       = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2: self._obs_buffer[0] = obs
            if i == self._skip - 1: self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)

In [6]:
class ClipRewardEnv(gym.RewardWrapper):
    def __init__(self, env):
        gym.RewardWrapper.__init__(self, env)

    def reward(self, reward):
        """Bin reward to {+1, 0, -1} by its sign."""
        return np.sign(reward)

In [7]:
class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env, width=84, height=84, grayscale=True, dict_space_key=None):
        """
        Warp frames to 84x84 as done in the Nature paper and later work.
        If the environment uses dictionary observations, `dict_space_key` can be specified which indicates which
        observation should be warped.
        """
        super().__init__(env)
        self._width = width
        self._height = height
        self._grayscale = grayscale
        self._key = dict_space_key
        if self._grayscale:
            num_colors = 1
        else:
            num_colors = 3

        new_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(self._height, self._width, num_colors),
            dtype=np.uint8,
        )
        if self._key is None:
            original_space = self.observation_space
            self.observation_space = new_space
        else:
            original_space = self.observation_space.spaces[self._key]
            self.observation_space.spaces[self._key] = new_space
        assert original_space.dtype == np.uint8 and len(original_space.shape) == 3

    def observation(self, obs):
        if self._key is None:
            frame = obs
        else:
            frame = obs[self._key]

        if self._grayscale:
            frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(
            frame, (self._width, self._height), interpolation=cv2.INTER_AREA
        )
        if self._grayscale:
            frame = np.expand_dims(frame, -1)

        if self._key is None:
            obs = frame
        else:
            obs = obs.copy()
            obs[self._key] = frame
        return obs

In [8]:
class FrameStack(gym.Wrapper):
    def __init__(self, env, k):
        """Stack k last frames.
        Returns lazy array, which is much more memory efficient.
        See Also
        --------
        baselines.common.atari_wrappers.LazyFrames
        """
        gym.Wrapper.__init__(self, env)
        self.k = k
        self.frames = deque([], maxlen=k)
        shp = env.observation_space.shape
        self.observation_space = spaces.Box(low=0, high=255, shape=(shp[:-1] + (shp[-1] * k,)), dtype=env.observation_space.dtype)

    def reset(self):
        ob = self.env.reset()
        for _ in range(self.k):
            self.frames.append(ob)
        return self._get_ob()

    def step(self, action):
        ob, reward, done, info = self.env.step(action)
        self.frames.append(ob)
        return self._get_ob(), reward, done, info

    def _get_ob(self):
        assert len(self.frames) == self.k
        return LazyFrames(list(self.frames))

In [9]:
class ScaledFloatFrame(gym.ObservationWrapper):
    def __init__(self, env):
        gym.ObservationWrapper.__init__(self, env)
        self.observation_space = gym.spaces.Box(low=0, high=1, shape=env.observation_space.shape, dtype=np.float32)

    def observation(self, observation):
        # careful! This undoes the memory optimization, use
        # with smaller replay buffers only.
        return np.array(observation).astype(np.float32) / 255.0

In [10]:
class LazyFrames(object):
    def __init__(self, frames):
        """This object ensures that common frames between the observations are only stored once.
        It exists purely to optimize memory usage which can be huge for DQN's 1M frames replay
        buffers.
        This object should only be converted to numpy array before being passed to the model.
        You'd not believe how complex the previous solution was."""
        self._frames = frames
        self._out = None

    def _force(self):
        if self._out is None:
            self._out = np.concatenate(self._frames, axis=-1)
            self._frames = None
        return self._out

    def __array__(self, dtype=None):
        out = self._force()
        if dtype is not None:
            out = out.astype(dtype)
        return out

    def __len__(self):
        return len(self._force())

    def __getitem__(self, i):
        return self._force()[i]

    def count(self):
        frames = self._force()
        return frames.shape[frames.ndim - 1]

    def frame(self, i):
        return self._force()[..., i]

In [11]:
def wrap_atari(env, max_episode_steps=None):
    assert 'NoFrameskip' in env.spec.id
    env = NoopResetEnv(env, noop_max=30)
    env = MaxAndSkipEnv(env, skip=4)

    assert max_episode_steps is None

    return env

In [12]:
def wrap_deepmind(env, episode_life=True, clip_rewards=True, frame_stack=False, scale=False):
    """Configure environment for DeepMind-style Atari.
    """
    if episode_life:
        env = EpisodicLifeEnv(env)
    if 'FIRE' in env.unwrapped.get_action_meanings():
        env = FireResetEnv(env)
    env = WarpFrame(env)
    if scale:
        env = ScaledFloatFrame(env)
    if clip_rewards:
        env = ClipRewardEnv(env)
    if frame_stack:
        env = FrameStack(env, 4)
    return env

In [13]:
class ImageToPyTorch(gym.ObservationWrapper):
    """
    Image shape to channels x weight x height
    """

    def __init__(self, env):
        super(ImageToPyTorch, self).__init__(env)
        old_shape = self.observation_space.shape
        self.observation_space = gym.spaces.Box(
            low=0,
            high=255,
            shape=(old_shape[-1], old_shape[0], old_shape[1]),
            dtype=np.uint8,
        )

    def observation(self, observation):
        return np.transpose(observation, axes=(2, 0, 1))

In [14]:
def wrap_pytorch(env):
    return ImageToPyTorch(env)

In [15]:
# Reference: https://arxiv.org/pdf/1707.06887.pdf
# https://github.com/ShangtongZhang/DeepRL/blob/master/deep_rl/agent/CategoricalDQN_agent.py

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter

In [16]:
import argparse
from distutils.util import strtobool
import collections
import numpy as np
import gym
from gym.wrappers import TimeLimit, Monitor
from gym.spaces import Discrete, Box, MultiBinary, MultiDiscrete, Space
import time
import random
import os

import matplotlib
matplotlib.use('Agg')
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from PIL import Image

In [17]:
exp_name = 'c51_atari' #the name of this experiment
gym_id = "BreakoutNoFrameskip-v4" #the id of the gym environment
learning_rate = 21e-5 #the learning rate of the optimizer
seed = 2 #seed of the experiment
total_timesteps = 100000 #total timesteps of the experiments
torch_deterministic = True #torch.backends.cudnn.deterministic
cuda = True #cuda will not be enabled
prod_model = False #run the script in production mode and use wandb to log outputs
capture_video = True #weather to capture videos of the agent performances (check out `videos` folder)

In [18]:
n_atoms = 51 #the number of atoms
v_min = -10 # the minimum of possible values
v_max = 10 # the maximum of possible values
buffer_size = 10000 #the replay memory buffer size
gamma = 0.99 #the discount factor gamma
target_network_frequency = 1000 #the timesteps it takes to update the target network
max_grad_norm = 0.5 #the maximum norm for the gradient clipping
batch_size = 32 #the batch size of sample from the reply memory
start_e = 1. #the starting epsilon for exploration
end_e = 0.01 #the ending epsilon for exploration
exploration_fraction = 0.10 #the fraction of `total-timesteps` it takes from start-e to go end-e
learning_starts = 80000 #timestep to start learning
train_frequency = 4 #the frequency of training

In [19]:
def image_return_distributions(pmfs, x_length, y_length, dpi=100):
    fig, ax = plt.subplots(figsize=(x_length/dpi,y_length/dpi), constrained_layout=True, dpi=100)
    current_palette = sns.color_palette(n_colors=env.action_space.n)
    df = pd.DataFrame(pmfs.T)
    for idx, y in enumerate(df.columns):
        ax.bar(np.linspace(v_min, v_max, num=n_atoms).astype(np.int), df[y], color=current_palette[idx])
    ax.set(xlabel='return distribution', ylabel='probs')
    fig.canvas.draw()
    X = np.array(fig.canvas.renderer.buffer_rgba())
    return_distribution_rgb_array = np.array(Image.fromarray(X).convert('RGB'))
    plt.close(fig)
    return return_distribution_rgb_array

In [20]:
def image_q_values(q_values, x_length, y_length, dpi=100):
    fig, ax = plt.subplots(figsize=(x_length/dpi,y_length/dpi), constrained_layout=True, dpi=100)
    df = pd.DataFrame(q_values.T)
    sns.barplot(x=df.index, y=0, data=df, ax=ax)
    ax.set(xlabel='actions', ylabel='q-values')
    fig.canvas.draw()
    X = np.array(fig.canvas.renderer.buffer_rgba())
    # Image.fromarray(X)
    q_value_rgb_array = np.array(Image.fromarray(X).convert('RGB'))
    plt.close(fig)
    return q_value_rgb_array

In [21]:
class QValueAndReturnDistributionVisualizationWrapper(gym.Wrapper):

    def __init__(self, env):
        super().__init__(env)
        self.env.reset()
        self.image_shape = self.env.render(mode="rgb_array").shape
        self.q_values = np.array([[0.,0.,0.,0.]])
        self.return_pmfs = np.zeros((4, 51))
        # self.metadata['video.frames_per_second'] = 60

    def set_q_values(self, q_values):
        self.q_values = q_values

    def set_return_pmfs(self, return_pmfs):
        self.return_pmfs = return_pmfs

    def render(self, mode="human"):
        if mode=="rgb_array":
            env_rgb_array = super().render(mode)
            self.q_value_rgb_array = image_q_values(
                self.q_values, self.image_shape[1], self.image_shape[0], dpi=100)
            self.return_rgb_array = image_return_distributions(
                self.return_pmfs, self.image_shape[1]*2, self.image_shape[0], dpi=100)
            return np.append(np.append(
                env_rgb_array,
                self.q_value_rgb_array, axis=1), self.return_rgb_array,
                axis=0)
        else:
            super().render(mode)

In [22]:
# modified from https://github.com/seungeunrho/minimalRL/blob/master/dqn.py#
class ReplayBuffer():
    def __init__(self, buffer_limit):
        self.buffer = collections.deque(maxlen=buffer_limit)
    
    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []
        
        for transition in mini_batch:
            s, a, r, s_prime, done_mask = transition
            s_lst.append(s)
            a_lst.append(a)
            r_lst.append(r)
            s_prime_lst.append(s_prime)
            done_mask_lst.append(done_mask)

        return np.array(s_lst), np.array(a_lst), \
               np.array(r_lst), np.array(s_prime_lst), \
               np.array(done_mask_lst)

In [23]:
experiment_name = f"{gym_id}__{exp_name}__{seed}__{int(time.time())}"
print(f"runs/{experiment_name}")

runs/BreakoutNoFrameskip-v4__c51_atari__2__1624518058


In [24]:
!pip install atari-py
!wget http://www.atarimania.com/roms/Roms.rar
!unrar x Roms.rar
!unzip ROMS.zip

--2021-06-24 07:01:01--  http://www.atarimania.com/roms/Roms.rar
Resolving www.atarimania.com (www.atarimania.com)... 195.154.81.199
Connecting to www.atarimania.com (www.atarimania.com)|195.154.81.199|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 11128004 (11M) [application/x-rar-compressed]
Saving to: ‘Roms.rar’

Roms.rar            100%[===================>]  10.61M   637KB/s    in 18s     

2021-06-24 07:01:19 (614 KB/s) - ‘Roms.rar’ saved [11128004/11128004]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Roms.rar

Extracting  HC ROMS.zip                                                   36%  OK 
Extracting  ROMS.zip                                                      74% 99%  OK 
All OK
Archive:  ROMS.zip
   creating: ROMS/
  inflating: ROMS/128 in 1 Game Select ROM (128 in 1) (Unknown) ~.bin  
  inflating: ROMS/2 Pak Special - Cavern Blaster, City War (1992) (HES) (773-867) (PAL).bin  
  infl

In [25]:
!ls
!python -m atari_py.import_roms ROMS

'HC ROMS.zip'   ROMS   Roms.rar   ROMS.zip   sample_data
copying king_kong.bin from ROMS/King Kong (1982) (Tigervision - Software Electronics Corporation, Karl T. Olinger - Teldec) (7-001 - 3.60001 VE) (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/king_kong.bin
copying tutankham.bin from ROMS/Tutankham (1983) (Parker Brothers, Dave Engman, Dawn Stockbridge) (PB5340) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/tutankham.bin
copying space_invaders.bin from ROMS/Space Invaders (1980) (Atari, Richard Maurer - Sears) (CX2632 - 49-75153) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/space_invaders.bin
copying frogger.bin from ROMS/Frogger (1982) (Parker Brothers, Ed English, David Lamkins) (PB5300) ~.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/frogger.bin
copying video_pinball.bin from ROMS/Video Pinball - Arcade Pinball (1981) (Atari, Bob Smith - Sears) (CX2648 - 49-75161) ~.bin to /usr/local/lib/p

In [35]:
device = torch.device('cuda' if torch.cuda.is_available() and cuda else 'cpu')
env = gym.make(gym_id)
env = wrap_atari(env)
env = gym.wrappers.RecordEpisodeStatistics(env) # records episode reward in `info['episode']['r']`
if capture_video:
    env = QValueAndReturnDistributionVisualizationWrapper(env)
    env = Monitor(env, f'videos/{experiment_name}')
env = wrap_pytorch(
    wrap_deepmind(
        env,
        clip_rewards=True,
        frame_stack=True,
        scale=False,
    )
)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = torch_deterministic
env.seed(seed)
env.action_space.seed(seed)
env.observation_space.seed(seed)
# respect the default timelimit
assert isinstance(env.action_space, Discrete), "only discrete action space is supported"

In [36]:
# ALGO LOGIC: initialize agent here:
# tricks taken from https://github.com/cpnota/autonomous-learning-library/blob/6d1111afce0d1582de463326f7d078a86e850551/all/presets/atari/models/__init__.py#L16
# apparently matters
class Linear0(nn.Linear):
    def reset_parameters(self):
        nn.init.constant_(self.weight, 0.0)
        if self.bias is not None:
            nn.init.constant_(self.bias, 0.0)

In [37]:
class Scale(nn.Module):
    def __init__(self, scale):
        super().__init__()
        self.scale = scale

    def forward(self, x):
        return x * self.scale

In [38]:
class QNetwork(nn.Module):
    def __init__(self, env, device, frames=4, n_atoms=51, v_min=-10, v_max=10):
        super(QNetwork, self).__init__()
        self.n_atoms = n_atoms
        self.atoms = torch.linspace(v_min, v_max, steps=n_atoms).to(device)
        self.network = nn.Sequential(
            Scale(1/255),
            nn.Conv2d(frames, 32, 8, stride=4),
            nn.ReLU(),
            nn.Conv2d(32, 64, 4, stride=2),
            nn.ReLU(),
            nn.Conv2d(64, 64, 3, stride=1),
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(3136, 512),
            nn.ReLU(),
            Linear0(512, env.action_space.n * n_atoms)
        )

    def forward(self, x, device):
        x = torch.Tensor(x).to(device)
        return self.network(x)

    def get_action(self, env, device, x, action=None):
        logits = self.forward(x, device)
        # probability mass function for each action
        pmfs = torch.softmax(logits.view(len(x), env.action_space.n, self.n_atoms), dim=2)
        q_values = (pmfs*self.atoms).sum(2)
        if action is None:
            action = torch.argmax(q_values, 1)
        return action, pmfs[torch.arange(len(x)), action], q_values, pmfs

In [39]:
def linear_schedule(start_e: float, end_e: float, duration: int, t: int):
    slope =  (end_e - start_e) / duration
    return max(slope * t + start_e, end_e)

In [40]:
rb = ReplayBuffer(buffer_size)
q_network = QNetwork(env, device, n_atoms=n_atoms, v_min=v_min, v_max=v_max).to(device)
target_network = QNetwork(env, device, n_atoms=n_atoms, v_min=v_min, v_max=v_max).to(device)
target_network.load_state_dict(q_network.state_dict())
optimizer = optim.Adam(q_network.parameters(), lr=learning_rate, eps=0.01/batch_size)
loss_fn = nn.MSELoss()
print(device.__repr__())
print(q_network)

# TRY NOT TO MODIFY: start the game
obs = env.reset()
episode_reward = 0
for global_step in tqdm.tqdm(range(total_timesteps)):
    # ALGO LOGIC: put action logic here
    epsilon = linear_schedule(start_e, end_e, exploration_fraction*total_timesteps, global_step)
    action, pmf, q_values, pmfs = q_network.get_action(env, device, obs.reshape((1,)+obs.shape))
    action = action.tolist()[0]
    if capture_video:
        env.set_q_values(np.array(q_values.tolist()))
        env.set_return_pmfs(np.array(pmfs.tolist())[0])
    if random.random() < epsilon:
        action = env.action_space.sample()

    next_obs, reward, done, info = env.step(action)
    episode_reward += reward

    # if 'episode' in info.keys():
    #     print(f"global_step={global_step}, episode_reward={info['episode']['r']}")
    #     writer.add_scalar("charts/episode_reward", info['episode']['r'], global_step)
    #     writer.add_scalar("charts/epsilon", epsilon, global_step)

    # ALGO LOGIC: training.
    rb.put((obs, action, reward, next_obs, done))
    if global_step > learning_starts and global_step % train_frequency == 0:
        s_obs, s_actions, s_rewards, s_next_obses, s_dones = rb.sample(batch_size)
        with torch.no_grad():
            _, next_pmfs, _, _ = target_network.get_action(env, device, s_next_obses)
            next_atoms = torch.Tensor(s_rewards).to(device).unsqueeze(-1) + gamma * q_network.atoms  * (1 - torch.Tensor(s_dones).to(device).unsqueeze(-1))
            delta_z = q_network.atoms[1]-q_network.atoms[0]
            tz = next_atoms.clamp(v_min, v_max)
            
            b = (tz - v_min)/ delta_z
            l = b.floor().clamp(0, n_atoms-1)
            u = b.ceil().clamp(0, n_atoms-1)
            d_m_l = (u + (l == u).float() - b) * next_pmfs
            d_m_u = (b - l) * next_pmfs
            target_pmfs = torch.zeros_like(next_pmfs)
            for i in range(target_pmfs.size(0)):
                target_pmfs[i].index_add_(0, l[i].long(), d_m_l[i])
                target_pmfs[i].index_add_(0, u[i].long(), d_m_u[i])
        
        _, old_pmfs, _, _ = q_network.get_action(env, device, s_obs, s_actions)
        loss = (-(target_pmfs * old_pmfs.clamp(min=1e-5).log()).sum(-1)).mean()
        
        # if global_step % 100 == 0:
        #     writer.add_scalar("losses/td_loss", loss, global_step)

        # optimize the midel
        optimizer.zero_grad()
        loss.backward()
        nn.utils.clip_grad_norm_(list(q_network.parameters()), max_grad_norm)
        optimizer.step()

        # update the target network
        if global_step % target_network_frequency == 0:
            target_network.load_state_dict(q_network.state_dict())

    obs = next_obs
    if done:
        # important to note that because `EpisodicLifeEnv` wrapper is applied,
        # the real episode reward is actually the sum of episode reward of 5 lives
        # which we record through `info['episode']['r']` provided by gym.wrappers.RecordEpisodeStatistics
        obs, episode_reward = env.reset(), 0

env.close()

device(type='cuda')
QNetwork(
  (network): Sequential(
    (0): Scale()
    (1): Conv2d(4, 32, kernel_size=(8, 8), stride=(4, 4))
    (2): ReLU()
    (3): Conv2d(32, 64, kernel_size=(4, 4), stride=(2, 2))
    (4): ReLU()
    (5): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (6): ReLU()
    (7): Flatten(start_dim=1, end_dim=-1)
    (8): Linear(in_features=3136, out_features=512, bias=True)
    (9): ReLU()
    (10): Linear0(in_features=512, out_features=204, bias=True)
  )
)


流式输出内容被截断，只能显示最后 5000 行内容。
100%|██████████| 100000/100000 [19:43<00:00, 84.47it/s]


In [ ]:
# please manually download video in the /video directory